In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import pandas as pd
from layers import conv_layer, max_pool_2x2, full_layer
import datetime

DATA_DIR = 'tmp/data'
MINIBATCH_SIZE = 50
STEPS = 5000

# mnist[0]은 이미지, mnist[1]은 숫자값 ont Hot
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [211]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)

# Loss Function과 Train
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Accuracy 평가
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [213]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        batch = mnist.train.next_batch(MINIBATCH_SIZE)
        train = sess.run([x,y_, x_image, conv1, conv1_pool, conv2, conv2_pool, conv2_flat, full_1, train_step], feed_dict={x: batch[0], y_: batch[1], keep_prob:0.5})
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1],
                                                           keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))
            #print('{} conv1 Dimension: {}'.format(i, train[3][0].shape))
            #print('conv1_pool Dimension: {}'.format(train[4][0].shape))
            #print('conv1 Dimension: {}'.format(train[5][0].shape))
            #print('conv1_pool Dimension: {}'.format(train[6][0].shape))
            print('conv2_flat Dimension: {}'.format(train[7][0].shape))
            print('full_1 Dimension: {}'.format(train[8][0].shape))


step 0, training accuracy 0.05999999865889549
conv2_flat Dimension: (3136,)
full_1 Dimension: (1024,)


In [212]:
# 완전 연결 계측에서 편향을 적용하지 않은 결과
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(STEPS):
        batch = mnist.train.next_batch(MINIBATCH_SIZE)
        train = sess.run([x,y_, x_image, conv1, conv1_pool, conv2, conv2_pool, conv2_flat, full_1, train_step], feed_dict={x: batch[0], y_: batch[1], keep_prob:0.5})
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1],
                                                           keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))
            #print('{} conv1 Dimension: {}'.format(i, train[3][0].shape))
            #print('conv1_pool Dimension: {}'.format(train[4][0].shape))
            #print('conv1 Dimension: {}'.format(train[5][0].shape))
            #print('conv1_pool Dimension: {}'.format(train[6][0].shape))
            #print('conv2_flat Dimension: {}'.format(train[7][0].shape))
            #print('full_1 Dimension: {}'.format(train[8][0].shape))


step 0, training accuracy 0.11999999731779099
step 100, training accuracy 0.8399999737739563
step 200, training accuracy 0.8399999737739563
step 300, training accuracy 0.9599999785423279
step 400, training accuracy 0.9200000166893005
step 500, training accuracy 0.9200000166893005
step 600, training accuracy 0.8799999952316284
step 700, training accuracy 1.0
step 800, training accuracy 0.9800000190734863
step 900, training accuracy 1.0
step 1000, training accuracy 1.0
step 1100, training accuracy 0.8399999737739563
step 1200, training accuracy 0.9800000190734863
step 1300, training accuracy 1.0
step 1400, training accuracy 1.0
step 1500, training accuracy 1.0
step 1600, training accuracy 0.9200000166893005
step 1700, training accuracy 1.0
step 1800, training accuracy 0.9599999785423279
step 1900, training accuracy 0.9800000190734863
step 2000, training accuracy 0.9599999785423279
step 2100, training accuracy 0.9599999785423279
step 2200, training accuracy 0.9599999785423279
step 2300, t

# mnist데이터 확인하기

In [40]:
image_cnt = 2
with tf.Session() as sess:
    batch = mnist.train.next_batch(image_cnt)
    print(type(batch[0]))
    batch_x_reshape = batch[0].reshape(image_cnt,28,28)
    batch_y = batch[1]
    print(batch_y[0])    

<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [43]:
x_df = pd.DataFrame(batch_x_reshape[0])
y = batch_y[0]
print(y)
x_df

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.380392,0.564706,0.564706,0.564706,0.282353,0.133333,0.133333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.882353,0.992157,0.992157,0.992157,0.992157,0.992157,0.992157,...,0.694118,0.694118,0.431373,0.262745,0.035294,0.0,0.0,0.0,0.0,0.0


In [41]:
x_df = pd.DataFrame(batch_x_reshape[0])
y = batch_y[0]
print(y)

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [37]:
x_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.192157,0.670588,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.388235,0.992157,...,0.643137,0.462745,0.188235,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.019608,0.650980,...,0.992157,0.992157,0.964706,0.635294,0.000000,0.000000,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.027451,...,0.992157,0.992157,0.992157,0.941177,0.000000,0.000000,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.992157,0.992157,0.992157,0.482353,0.000000,0.000000,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.992157,0.992157,0.925490,0.054902,0.000000,0.000000,0.0,0.0,0.0,0.0
